In [1]:
import os
import sys
import json
import numpy as np
import time
from PIL import Image, ImageDraw
from skimage.io import imsave
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Code to Test the Mask_RCNN on WAD Test and Validation Dataset


ROOT_DIR = './'
assert os.path.exists(ROOT_DIR), 'ROOT_DIR does not exist.'

# Import mrcnn libraries
sys.path.append(ROOT_DIR) 
from mrcnn.config import Config
import mrcnn.utils as utils
from mrcnn import visualize
import mrcnn.model as modellib

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR,"Pretrained_Model","mask_rcnn_coco.h5")

# ## Configuration
# Define configurations for training on the Kaggle WAD Dataset

class WADConfig(Config):
    """Configuration for training on the cigarette butts dataset.
    Derives from the base Config class and overrides values specific
    to the cigarette butts dataset.
    """
    # Give the configuration a recognizable name
    NAME = "WAD"

    # Train on 1 GPU and 1 image per GPU. Batch size is 1 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 5

    # Number of classes (including background)
    NUM_CLASSES = 1 + 7  # background + 1 (7 classes in our dataset)

    # All of our training images are 512x512
    IMAGE_MIN_DIM = 640
    IMAGE_MAX_DIM = 832

    # You can experiment with this number to see if it improves training
    STEPS_PER_EPOCH = 10

    # This is how often validation is run. If you are using too much hard drive space
    # on saved models (in the MODEL_DIR), try making this value larger.
    VALIDATION_STEPS = 5
    
    # Can also use resnet101
    BACKBONE = 'resnet50'

    # Set Region Proposal Anchor Scales
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)
    TRAIN_ROIS_PER_IMAGE = 32
    MAX_GT_INSTANCES = 50 
    POST_NMS_ROIS_INFERENCE = 500 
    POST_NMS_ROIS_TRAINING = 1000 
    
config = WADConfig()
config.display()


# # Define the dataset
# Generic COCO Like Dataset Loader Class

class CocoLikeDataset(utils.Dataset):
    """ Generates a COCO-like dataset, i.e. an image dataset annotated in the style of the COCO dataset.
        See http://cocodataset.org/#home for more information.
    """
    def load_data(self, annotation_json, images_dir):
        """ Load the coco-like dataset from json
        Args:
            annotation_json: The path to the coco annotations json file
            images_dir: The directory holding the images referred to by the json file
        """
        # Load json from file
        json_file = open(annotation_json)
        coco_json = json.load(json_file)
        json_file.close()
        
        # Add the class names using the base method from utils.Dataset
        source_name = "coco_like"
        for category in coco_json['categories']:
            class_id = category['id']
            class_name = category['name']
            if class_id < 1:
                print('Error: Class id for "{}" cannot be less than one. (0 is reserved for the background)'.format(class_name))
                return
            
            self.add_class(source_name, class_id, class_name)
        
        # Get all annotations
        annotations = {}
        for annotation in coco_json['annotations']:
            image_id = annotation['image_id']
            if image_id not in annotations:
                annotations[image_id] = []
            annotations[image_id].append(annotation)
        
        # Get all images and add them to the dataset
        seen_images = {}
        for image in coco_json['images']:
            image_id = image['id']
            if image_id in seen_images:
                print("Warning: Skipping duplicate image id: {}".format(image))
            else:
                seen_images[image_id] = image
                try:
                    image_file_name = image['file_name']
                    image_width = image['width']
                    image_height = image['height']
                except KeyError as key:
                    print("Warning: Skipping image (id: {}) with missing key: {}".format(image_id, key))
                
                image_path = os.path.abspath(os.path.join(images_dir, image_file_name))
                image_annotations = annotations[image_id]
                
                # Add the image using the base method from utils.Dataset
                self.add_image(
                    source=source_name,
                    image_id=image_id,
                    path=image_path,
                    width=image_width,
                    height=image_height,
                    annotations=image_annotations
                )
                
    def load_mask(self, image_id):
        """ Load instance masks for the given image.
        MaskRCNN expects masks in the form of a bitmap [height, width, instances].
        Args:
            image_id: The id of the image to load masks for
        Returns:
            masks: A bool array of shape [height, width, instance count] with
                one mask per instance.
            class_ids: a 1D array of class IDs of the instance masks.
        """
        image_info = self.image_info[image_id]
        annotations = image_info['annotations']
        instance_masks = []
        class_ids = []
        
        for annotation in annotations:
            class_id = annotation['category_id']
            mask = Image.new('1', (image_info['width'], image_info['height']))
            mask_draw = ImageDraw.ImageDraw(mask, '1')
            for segmentation in annotation['segmentation']:
                mask_draw.polygon(segmentation, fill=1)
                bool_array = np.array(mask) > 0
                instance_masks.append(bool_array)
                class_ids.append(class_id)

        mask = np.dstack(instance_masks)
        class_ids = np.array(class_ids, dtype=np.int32)
        
        return mask, class_ids


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     5
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 5
IMAGE_MAX_DIM                  832
IMAGE_META_SIZE                20
IMAGE_MIN_DIM                  640
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [832 832   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES 

Using TensorFlow backend.


In [3]:
#dataset_train = CocoLikeDataset()
#dataset_train.load_data('/home/ayush/Instance_Segmentation/all/Sample_Dataset/wad_sample_train.json', '/home/ayush/Instance_Segmentation/all/Sample_Dataset/sample_train_color')
#dataset_train.prepare()

dataset_val = CocoLikeDataset()
dataset_val.load_data('/home/ayush/Instance_Segmentation/all/Sample_Dataset/wad_sample_val.json', '/home/ayush/Instance_Segmentation/all/Sample_Dataset/sample_val_color')
dataset_val.prepare()

In [4]:
# # Prepare to run Inference

class InferenceConfig(WADConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    IMAGE_MIN_DIM = 640
    IMAGE_MAX_DIM = 832
    DETECTION_MIN_CONFIDENCE = 0.9
    

inference_config = InferenceConfig()


# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)


# Get path to saved weights
# Either set a specific path or find last trained weights
model_path = os.path.join(ROOT_DIR, ".h5 file name here")
#model_path = model.find_last()

model_path = './logs/wad20190311T1356/mask_rcnn_wad_0078.h5'

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Loading weights from  ./logs/wad20190311T1356/mask_rcnn_wad_0078.h5
Re-starting from epoch 78


In [7]:
import pdb
# # Run Inference

import skimage
real_test_dir = '/home/ayush/Instance_Segmentation/all/Sample_Dataset/sample_val_color/'
image_paths = []
for filename in os.listdir(real_test_dir):
    if os.path.splitext(filename)[1].lower() in ['.png', '.jpg', '.jpeg']:
        image_paths.append(os.path.join(real_test_dir, filename))
        
image_paths.sort()
#image_paths = image_paths[2000:2001]
#print(image_paths)

id2classid = {1:33,
              2:34,
              3:35,
              4:36,
              5:38,
              6:39,
              7:40,}

for image_path in image_paths:
    
    im_name = os.path.basename(image_path).split('.')[0]
    #print(im_name)

    img = skimage.io.imread(image_path)
    img_arr = np.array(img)
    results = model.detect([img_arr], verbose=0)
    r = results[0]
    mask = np.zeros((r['masks'].shape[0:2]))
    for pointer, _id in enumerate(r['class_ids']):
        object_id = id2classid[_id]
        mask += r['masks'][:, :, pointer]*object_id*1000
    save_path = '../Sample_Dataset/sample_label_val_pred/'+im_name+'.png'
    imsave(save_path, np.ndarray.astype(mask, np.int32))
        
    #result_im = visualize.display_instances(img, r['rois'], r['masks'], r['class_ids'], dataset_val.class_names, r['scores'], figsize=(5,5))
    #if np.min(result_im)==-1:
    #    continue
    #break
    #save_path = '../Sample_Dataset/sample_label_val_pred/'+im_name+'.png'
    #print(type(result_im))
    #print(save_path)
    #imsave(save_path,result_im*255)